# Components

> Components that are the building blocks to the UI

## Imports

In [ ]:
#| default_exp components

In [ ]:
#| hide
#| export
from fasthtml.common import *
from fasthtml.svg import Svg
from enum import Enum, EnumType
from fasthtml.components import Uk_select,Uk_input_tag
from functools import partial
from itertools import zip_longest

In [ ]:
from nbdev.showdoc import show_doc
from IPython.display import HTML

## Utils

### Tests

In [ ]:
from fastcore.test import *

In [ ]:
def test_xml_eq(a, b):
    "Tests if if 2 FT components generate the same xml"
    test_eq(*map(lambda x: x if isinstance(x,str) else to_xml(x), (a, b)))

### Other

In [ ]:
#| export
# need a better name, stringify might be too general for what it does 
def stringify(o # String, Tuple, or Enum options we want stringified
             ): # String that can be passed FT comp args (such as `cls=`)
    "Converts input types into strings that can be passed to FT components"  
    if is_listy(o): return ' '.join(map(str,o)) if o else ""
    return o.__str__()

In [ ]:
show_doc(stringify)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L24){target="_blank" style="float:right; font-size:smaller"}

### stringify

>      stringify (o)

*Converts input types into strings that can be passed to FT components*

|    | **Details** |
| -- | ----------- |
| o | String, Tuple, or Enum options we want stringified |

In [ ]:
assert stringify('abc') == 'abc'
assert stringify(('abc','def')) == 'abc def'
assert 'uk-input ' + stringify(()) == 'uk-input '
assert 'uk-input ' + stringify("") == 'uk-input '

In [ ]:
#| export
class VEnum(Enum):
    def __add__(self, other):
        "Add other enums, listy, or strings"
        return stringify((self, other))

    def __radd__(self, other):
        "Add other enums, listy, or strings"
        return stringify((other, self))
    
    def __str__(self):
        "Stringifies with uk-{attr}-{value} format"
        base = self.__class__.__name__       
        if isinstance(self.__class__, EnumType):
            base = base.lstrip('Uk').rstrip('T')
        return f"uk-{base.lower()}-{self.value}".strip('-')

## Enums

### Theme

You can select a theme color to and get all the headers.  This brings in the most standard grouping:
+ Tailwind: Optional, but typically FrankenUI is used in conjunction with tailwind
+ Uikit: Many things are styled with uikit
+ UIkit icons: Gives many icons OOTB, and is nice to have handy.
+ Franken WC: Neccesarry for some of the (minimal) interactivity components have

In [ ]:
#| export
class Theme(Enum):
    slate = "slate"
    stone = "stone"
    gray = "gray"
    neutral = "neutral"
    red = "red"
    rose = "rose"
    orange = "orange"
    green = "green"
    blue = "blue"
    yellow = "yellow"
    violet = "violet"
    zinc = "zinc"

    def headers(self):
        js = (Script(src="https://cdn.tailwindcss.com"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"),
              Script(type="module", src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js")
              )
        _url = f"https://unpkg.com/franken-wc@0.0.6/dist/css/{self.value}.min.css"
        return (*js, Link(rel="stylesheet", href=_url))

In [ ]:
hdrs = Theme.blue.headers()
app = FastHTML(hdrs=hdrs)
hdrs

(script(('',),{'src': 'https://cdn.tailwindcss.com'}),
 script(('',),{'src': 'https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js'}),
 script(('',),{'src': 'https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js'}),
 script(('',),{'type': 'module', 'src': 'https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js'}),
 link((),{'rel': 'stylesheet', 'href': 'https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css'}))

### Style

#### `B` (Base)

Base enums all will end with `B`.  These are designed to make classes discoverable in python.  These should be generally a 1:1 mapping to a class.

These are the lowest level class selection options without dropping into strings.  We don't need to cover everything, but we want most useful things in here.

In [ ]:
#| export
class TextB(Enum):
    sz_xsmall = 'text-xs'
    sz_small = 'text-sm'
    sz_medium = 'text-base'
    sz_large = 'text-lg'
    cl_muted = 'uk-text-muted'
    
    wt_light = 'font-light'
    wt_normal = 'font-normal'
    wt_medium = 'font-medium'
    wt_bold = 'font-bold'
# font-thin	font-weight: 100;
# font-extralight	font-weight: 200;
# font-light	font-weight: 300;
# font-normal	font-weight: 400;
# 
# font-semibold	font-weight: 600;
# font-bold	font-weight: 700;
# font-extrabold	font-weight: 800;
# font-black    
    
    
# font-medium text-sm
    def __str__(self):
        return self.value

### `T` (Transform)

Transform enums all will end with `T`.  These are designed to make useful class groups discoverable in python.

Each option should be a group of enums that are useful in many cases.  For example `muted_sm` is very commonly used as a subheader below a card title and looks great.  These should be high level things that users can generally just pick from to have a decent default without much thought.

In [ ]:
#| export
class TextT(Enum):
    muted_xs = TextB.sz_xsmall, TextB.cl_muted 
    muted_sm = TextB.sz_small, TextB.cl_muted # Text below card headings
    muted_med = TextB.sz_medium, TextB.cl_muted 
    muted_lg = TextB.sz_large, TextB.cl_muted 
    medium_sm = TextB.sz_small, TextB.wt_medium
    medium_xs = TextB.sz_xsmall, TextB.wt_medium

    def __str__(self):
        if is_listy(self.value): return ' '.join(map(str,self.value))
        return self.value

In [ ]:
type(TextT.muted_sm)

<enum 'TextT'>

## Base Components

### Icons and Avatars

In [ ]:
#| export
def UkIcon(icon, ratio=1,cls=()):
    return Span(uk_icon=f"icon: {icon}; ratio: {ratio}",cls=cls)

In [ ]:
#| export
def DiceBearAvatar(seed_name, h, w):
    url = 'https://api.dicebear.com/8.x/lorelei/svg?seed='
    return Span(cls=f"relative flex h-{h} w-{w} shrink-0 overflow-hidden rounded-full bg-accent")(
            Img(cls="aspect-square h-full w-full", alt="Avatar", src=f"{url}{seed_name}"))
    

In [ ]:
#| export
def NavP(*c, cls=TextT.muted_sm): return P(cls=cls)(*c)

def SpacedPP(left, right=None):
    return FullySpacedContainer(NavP(left),NavP(right) if right else '')

def SpacedPPs(*c):
    return [SpacedPP(*tuplify(o)) for o in c]

def SpacedTxtIcon(txt, icon, ratio, icon_right=True):
    c = (NavP(txt),UkIcon(icon,ratio))
    if not icon_right: c = reversed(c)
    return FullySpacedContainer(*c)  

def LAlignedTxtIcon(txt, icon='play-circle', gap=2, ratio=1, icon_right=True, txt_cls=None):
    # TODO: Test and make sure this works
    # TODO: How do you pass text with a color? Should txt be a string?  Or should you pass a P?  
    c = (txt if isinstance(txt, FT) else NavP(txt,cls=ifnone(txt_cls,TextT.muted_sm)),UkIcon(icon,ratio))
    if not icon_right: c = reversed(c)
    return Div(cls=f'flex items-center space-x-{gap}')(*c)

def RAlignedTxtIcon(txt, icon, gap=2, ratio=1, txt_cls=None):
    return LAlignedTxtIcon(txt, icon, gap=gap, ratio=ratio, txt_cls=txt_cls, icon_right=False)

### Spacing 

In [ ]:
#|export
def Grid(*c, cols=3, gap=2, cls=(), **kwargs):
    cls = stringify(cls)
    return Div(cls=f'grid grid-cols-{cols} gap-{gap} '+cls, **kwargs)(*c)

In [ ]:
#| export
def FullySpacedContainer(*c,wrap_tag=None):
    return Div(cls='uk-flex uk-flex-between uk-flex-middle uk-width-1-1')(*(map(ifnone(wrap_tag,noop),c)))

In [ ]:
#| export
def CenteredContainer(*c,cls=()):
    return Div(cls='flex flex-col items-center justify-center ' + stringify(cls))(*c)

### Inputs

In [ ]:
#| export
from typing import Union, Tuple, Optional
from fastcore.all import L

def UkGenericInput(input_fn: FT, # FT Components that generates a user input (e.g. `TextArea`)
                    label:str|FT=(), # String or FT component that goes in `Label`
                    lbl_cls:str|Enum=(), # Additional classes that goes in `Label`
                    inp_cls:str|Enum=(), # Additional classes that go in user input (e.g. `TextArea`)
                    cls:str|Enum=('space-y-2',), # Div cls
#                     id: str="", # ID of the user input (e.g. `TextArea`)
                   **kwargs # Passed to `input_fn` (e.g. ` TextArea`)
                  ) -> FT: # FT component in structure `(Div(label,input))`
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UKTextArea` which is created for you in this library"
    lbl_cls, inp_cls, cls = map(stringify,(lbl_cls, inp_cls, cls))
    if label:  label = Label(cls='uk-form-label '+lbl_cls)(label)
    if label and id: label.attrs['for'] = id
    res = input_fn(**kwargs)
    if inp_cls: res.attrs['class'] += inp_cls
    return Div(cls=cls)(label, res)

In [ ]:
show_doc(UkGenericInput)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L160){target="_blank" style="float:right; font-size:smaller"}

### UkGenericInput

>      UkGenericInput (input_fn:fastcore.xml.FT, label:str|fastcore.xml.FT=(),
>                      lbl_cls:str|enum.Enum=(), inp_cls:str|enum.Enum=(),
>                      cls:str|enum.Enum=('space-y-2',), **kwargs)

*`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UKTextArea` which is created for you in this library*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| input_fn | FT |  | FT Components that generates a user input (e.g. `TextArea`) |
| label | str \| fastcore.xml.FT | () | String or FT component that goes in `Label` |
| lbl_cls | str \| enum.Enum | () | Additional classes that goes in `Label` |
| inp_cls | str \| enum.Enum | () | Additional classes that go in user input (e.g. `TextArea`) |
| cls | str \| enum.Enum | ('space-y-2',) | Div cls |
| kwargs |  |  |  |
| **Returns** | **FT** |  | **FT component in structure `(Div(label,input))`** |

In [ ]:
#| export
UkInput =     partial(UkGenericInput, partial(Input, cls='uk-input'))
UkSwitch =    partial(UkGenericInput, partial(CheckboxX,    cls='uk-toggle-switch uk-toggle-switch-primary')) 
UkTextArea =  partial(UkGenericInput, partial(Textarea,     cls='uk-textarea'))
UkFormLabel = partial(UkGenericInput, partial(Uk_input_tag, cls='uk-form-label'))


In [ ]:
show_doc(UkInput,title_level=4)

---

#### partial(<function ft_hx>, 'input', cls='uk-input'))

>      partial(<function ft_hx>, 'input', cls='uk-input'))
>              (label:str|fastcore.xml.FT=(), lbl_cls:str|enum.Enum=(),
>              inp_cls:str|enum.Enum=(), cls:str|enum.Enum=('space-y-2',),
>              **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [ ]:
UkInput("MyLabel",lbl_cls=TextT.muted_sm, inp_cls='font-bold',  id='myid')

```html
<div class="space-y-2">
<label class="uk-form-label text-sm uk-text-muted" for="<built-in function id>">MyLabel</label>  <input id="myid" class="uk-inputfont-bold" name="myid">
</div>

```

In [ ]:
show_doc(UkSwitch,title_level=4)

---

#### partial(<function CheckboxX>, cls='uk-toggle-switch uk-toggle-switch-primary'))

>      partial(<function CheckboxX>, cls='uk-toggle-switch uk-
>              toggle-switch-primary')) (label:str|fastcore.xml.FT=(),
>              lbl_cls:str|enum.Enum=(), inp_cls:str|enum.Enum=(),
>              cls:str|enum.Enum=('space-y-2',), **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [ ]:
show_doc(UkTextArea,title_level=4)

---

#### partial(<function ft_hx>, 'textarea', cls='uk-textarea'))

>      partial(<function ft_hx>, 'textarea', cls='uk-textarea'))
>              (label:str|fastcore.xml.FT=(), lbl_cls:str|enum.Enum=(),
>              inp_cls:str|enum.Enum=(), cls:str|enum.Enum=('space-y-2',),
>              **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [ ]:
show_doc(UkFormLabel,title_level=4)

---

#### _f at 0x112e88a40>, cls='uk-form-label'))

>      _f at 0x112e88a40>, cls='uk-form-label')) (label:str|fastcore.xml.FT=(),
>                                                 lbl_cls:str|enum.Enum=(),
>                                                 inp_cls:str|enum.Enum=(), cls:
>                                                 str|enum.Enum=('space-y-2',),
>                                                 **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [ ]:
#| export
def Options(*c, # Content for an `Option`
            selected_idx=None # Index location of selected `Option`
           ):
    "Generates list of `Option`s with the proper `selected_idx`"
    return [Option(o,selected=i==selected_idx) for i,o in enumerate(c)]

In [ ]:
#| export
def UkSelect(*options,
             label=(),
             lbl_cls=(),
             inp_cls=(),
             cls=('space-y-2',),
             id="",
             name="",
             placeholder="",
             searchable=False,
             **kwargs):
    lbl_cls, inp_cls, cls = map(stringify, (lbl_cls, inp_cls, cls))
    if label:
        lbl = Label(cls=f'uk-form-label {lbl_cls}', fr=id)(label) if id else Label(cls=f'uk-form-label {lbl_cls}')(label)
    select = Uk_select(cls=inp_cls, uk_cloak=True, id=id, name=name, placeholder=placeholder, searchable=searchable, **kwargs)
    select = select(*options)
    return Div(cls=cls)(lbl, select) if label else Div(cls=cls)(select)

In [ ]:
show_doc(UkSelect)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L191){target="_blank" style="float:right; font-size:smaller"}

### UkSelect

>      UkSelect (*options, label=(), lbl_cls=(), inp_cls=(), cls=('space-y-2',),
>                id='', name='', placeholder='', searchable=False, **kwargs)

In [ ]:
# UkSelect(*_opts, "MyLabel",lbl_cls=TextT.muted_sm, inp_cls='font-bold',  id='myid')

In [ ]:
#| export
class UkButtonT(VEnum):
    default = 'default'
    primary = 'primary'
    secondary = 'secondary'
    danger = 'danger'
    ghost = 'ghost'
    text = 'text'
    link = 'link'

In [ ]:
#| export

def process_options(opts, hdrs):
    for i, (opt, hdr) in enumerate(zip_longest(opts, hdrs or [])):
        if hdr and len(hdr) > 0: yield Li(cls="uk-nav-header")(hdr if isinstance(hdr, FT) else Div(hdr))
        if isinstance(opt, (list, tuple)): yield from list(map(Li, opt))
        else: yield Li(opt)
        if i < len(opts) - 1:
            next_hdr = hdrs[i+1] if hdrs and i+1 < len(hdrs) else None
            if not next_hdr or len(next_hdr) == 0: yield Li(cls='uk-nav-divider')

def UkDropdownButton(
    options,        # List of options to be displayed in the dropdown
    option_hdrs=None,  # List of headers for each option group, or None
    label=None,     # String, FT component, or None for the `Button`
    btn_cls=UkButtonT.default,  # Button class(es)
    cls=(),         # Parent div class
    dd_cls=(),      # Class that goes on the dropdown container
    icon='triangle-down',  # Icon to use for the dropdown
    icon_cls='',    # Additional classes for the icon
    icon_position='right'  # Position of the icon: 'left' or 'right'
):
    dd_cls, btn_cls, cls, icon_cls = map(stringify, (dd_cls, btn_cls, cls, icon_cls))
    icon_component = UkIcon(icon, cls=icon_cls) if icon else None
    btn_content = [] if label is None else [label]
    if icon_component: btn_content.insert(0 if icon_position == 'left' else len(btn_content), icon_component)
    btn = Button(type='button', cls='uk-button ' + btn_cls)(*btn_content)
    dd = Div(uk_drop='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop ' + dd_cls)(
        Ul(cls='uk-dropdown-nav')(*process_options(options, option_hdrs))
    )
    return Div(cls=cls)(Div(cls='flex items-center space-x-4')(btn, dd))

In [ ]:
show_doc(UkDropdownButton)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L228){target="_blank" style="float:right; font-size:smaller"}

### UkDropdownButton

>      UkDropdownButton (options, option_hdrs=None, label=None,
>                        btn_cls=<UkButtonT.default: 'default'>, cls=(),
>                        dd_cls=(), icon='triangle-down', icon_cls='',
>                        icon_position='right')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| options |  |  | List of options to be displayed in the dropdown |
| option_hdrs | NoneType | None | List of headers for each option group, or None |
| label | NoneType | None | String, FT component, or None for the `Button` |
| btn_cls | UkButtonT | uk-button-default | Button class(es) |
| cls | tuple | () | Parent div class |
| dd_cls | tuple | () | Class that goes on the dropdown container |
| icon | str | triangle-down | Icon to use for the dropdown |
| icon_cls | str |  | Additional classes for the icon |
| icon_position | str | right | Position of the icon: 'left' or 'right' |

### Other

In [ ]:
#| export
def UkButton(*c, 
            cls=UkButtonT.default, # Use UkButtonT or styles 
            **kwargs):    
    return Button(type='button', cls='uk-button ' + stringify(cls), **kwargs)(*c)

In [ ]:
#| export
def UkGenericComponent(component_fn, *c, cls=(), **kwargs):
    res = component_fn(**kwargs)(*c)
    if cls: res.attrs['class'] += ' ' + cls
    return res

UkH1 = partial(UkGenericComponent, partial(H1,cls='uk-h1'))
UkH2 = partial(UkGenericComponent, partial(H2,cls='uk-h2'))
UkH3 = partial(UkGenericComponent, partial(H3,cls='uk-h3'))
UkH4 = partial(UkGenericComponent, partial(H4,cls='uk-h4'))
UkH5 = partial(UkGenericComponent, partial(H5,cls='uk-h5'))
UkH6 = partial(UkGenericComponent, partial(H6,cls='uk-h6'))


In [ ]:
UkH1("My Uk styled header"), UkH1('My Uk styled header',cls='another-class')

(h1(('My Uk styled header',),{'class': 'uk-h1'}),
 h1(('My Uk styled header',),{'class': 'uk-h1 another-class'}))

In [ ]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))

def UkHLine(lwidth=2, y_space=4): return Div(cls=f"my-{y_space} h-[{lwidth}px] w-full bg-secondary")

In [ ]:
#| export
def UkNavDivider(): return Li(cls="uk-nav-divider")

## Higher order components

### Navigation

#### Navbar

In [ ]:
#| export
def UkNavbarDropdown(*c, label, has_header=False):
    flattened = []
    for i, item in enumerate(c):
        if i > 0: flattened.append(Li(cls="uk-nav-divider"))
        if isinstance(item, (list,tuple)): flattened.extend(map(Li, item))
        else: flattened.append(Li(item, cls="uk-nav-header" if i == 0 and has_header else None))
    return Li(A(label, href='#'), Div(cls='uk-navbar-dropdown')(
        Ul(cls='uk-nav uk-navbar-dropdown-nav')(*flattened)))

In [ ]:
#| export

def _NavBarSide(n, s):
    def add_class(item):
        if isinstance(item, str): return Li(cls='uk-navbar-item')(item)
        else: item.attrs['class'] = f"{item.attrs.get('class', '')} uk-navbar-item".strip()
        return item
    return Div(cls=f'uk-navbar-{s}')(Ul(cls='uk-navbar-nav')(*map(add_class, tuplify(n))))

def UkNavbar(lnav: Sequence[Union[str, FT]]=None, 
             rnav: Sequence[Union[str, FT]]=None, 
             cls='z-10') -> FT:
    return Div(cls='uk-navbar-container uk-width-1-1 relative z-50 '+ cls, uk_navbar=True)(
             _NavBarSide(lnav,'left') if lnav else '',
             _NavBarSide(rnav,'right') if rnav else '')

#### Navtab

In [ ]:
#| export

def NavTab(text, active=False):
    return Li(cls="uk-active" if active else " ")(A(text, href="#demo", uk_toggle=True))

def UkTab(*items):
    return Ul(cls="uk-tab-alt max-w-96")(*[NavTab(item, active=i==0) for i, item in enumerate(items)])

#### Sidebar

In [ ]:
#| export

def UkSidebarItem(item, is_header=False): return UkH4(item) if is_header else A(role='button')(item)

def UkSidebarUl(*lis, cls='', **kwargs): 
    return Ul(cls=f"uk-nav uk-nav-secondary space-y-2 {cls}", **kwargs)(*map(Li,lis))

def UkSidebarSection(items, header=None, cls='', **kwargs):
    section = [UkSidebarItem(item) for item in items]
    if header: section.insert(0, UkSidebarItem(header, is_header=True))
    return UkSidebarUl(*section, cls=cls, **kwargs)

def UkSidebar(sections, headers=None, outer_margin=4, inner_margin=4, cls=(), **kwargs):
    headers = headers or [None] * len(sections)
    sidebar_content = map(lambda s_h: UkSidebarSection(*s_h, **kwargs), zip(sections, headers))
    return Div(cls=f"space-y-{inner_margin} p-{outer_margin} {cls}")(*sidebar_content)

### Cards

In [ ]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-card-header ' + header_cls)(header),]
    res += [Div(cls='uk-card-body ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-card-footer ' + footer_cls)(footer),]
    return Div(cls='uk-card '+cls, **kwargs)(*res)

### Modals

In [ ]:
#| export
def UkModalTitle(*c, cls=()): return Div(cls='uk-modal-title ' + stringify(cls))(*c)

def Modal(*c,
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls='p-6', # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-modal-header ' + header_cls)(header),]
    res += [Div(cls='uk-modal-body uk-modal-dialog ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-modal-footer ' + footer_cls)(footer),]
    return Div(cls='uk-modal uk-modal-container' + cls, uk_modal=True, **kwargs)(*res)


### Tables

In [ ]:
#| export
def default_header(col): return Th(cls='p-2')(col)
def default_cell(col, row): return Td(row[col], cls='p-2')

def header_row(columns, header_render):
    rndr = header_render or default_header
    return Tr(*map(rndr, columns))

def data_row(row, columns, cell_render):
    rndr = cell_render or default_cell
    return Tr(*[rndr(col, row) for col in columns])

def UkTable(columns, data, *args, cls=(), footer=None, cell_render=None, header_render=None, **kwargs):
    table_cls = 'uk-table uk-table-middle uk-table-divider uk-table-hover uk-table-small ' + stringify(cls)
    table_content = [
        Thead(header_row(columns, header_render)),
        Tbody(*map(lambda d: data_row(d, columns, cell_render), data))
    ]
    if footer: table_content.append(Tfoot(footer))
    return Table(cls=table_cls, *args, **kwargs)(*table_content)

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()